In [6]:
import numpy as np
import pandas as pd
from collections import Counter
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import spacy

In [19]:
path2files="/Users/ashishkumar/cs598dmcs/dataset/Hygiene/"
datFile = path2files+"hygiene.dat"
labelFile = path2files+"hygiene.dat.labels"
addDataFile = path2files + "hygiene.dat.additional"

df1 = pd.read_csv(addDataFile,header=None)
df1.columns = ['cuisine','zipCode','reviewCount','avgRating']
df2 = pd.read_csv(datFile, sep='\n',header=None)
#only first 546 restaurants are labeled
dataLabels = pd.read_csv(labelFile, sep='\n',header=None)
labeledData = 546
lbl = dataLabels[0:labeledData][0]

In [20]:
Counter(lbl)

Counter({'0': 273, '1': 273})

In [21]:
#df2.iloc[7][0]
trainigdf = df1[0:labeledData]
trainigdf['reviewText'] = df2[0:labeledData]
trainigdf['lbl'] = lbl


/Users/ashishkumar/anaconda3/envs/idp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ashishkumar/anaconda3/envs/idp/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [53]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(trainigdf['reviewText'])
sequences = tokenizer.texts_to_sequences(trainigdf['reviewText'])
data = pad_sequences(sequences, maxlen=600)


In [54]:
len(data)

546

In [57]:

'''
In line two, we add an Embedding layer. This layer lets the network expand each token to a larger vector, 
allowing the network to represent words in a meaningful way. We pass 10000 as the first argument, 
which is the size of our vocabulary (remember, we told the tokenizer to only use the 
10000 most common words earlier), and 512 as the second, which means that each token can be 
expanded to a vector of size 512. We give it an input_length of 600, 
which is the length of each of our sequences.
'''
model = Sequential()
model.add(Embedding(10000, 512, input_length=600))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
'''
slow to train
'''
model.fit(data, np.array(trainigdf['lbl']), validation_split = 0.5, epochs = 5 )

Train on 273 samples, validate on 273 samples
Epoch 1/5
273/273 [==============================] - 101s 369ms/step - loss: 0.6935 - acc: 0.5092 - val_loss: 0.6929 - val_acc: 0.5128
Epoch 2/5
273/273 [==============================] - 95s 349ms/step - loss: 0.7050 - acc: 0.8132 - val_loss: 0.6949 - val_acc: 0.5092
Epoch 3/5
273/273 [==============================] - 92s 336ms/step - loss: 0.5682 - acc: 0.8498 - val_loss: 0.7032 - val_acc: 0.5018
Epoch 4/5
273/273 [==============================] - 92s 338ms/step - loss: 0.4074 - acc: 0.9048 - val_loss: 0.7725 - val_acc: 0.4908
Epoch 5/5
273/273 [==============================] - 94s 343ms/step - loss: 0.2119 - acc: 0.9707 - val_loss: 0.8641 - val_acc: 0.4908


In [56]:
'''
By adding a CNN layer before the LSTM, we allow the LSTM to see sequences of chunks instead of sequences of words. 
'''
model = Sequential()
model.add(Embedding(20000, 512, input_length=600))
model.add(Dropout(0.2))
model.add(Conv1D(16, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(512))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data, np.array(trainigdf['lbl']), validation_split = 0.5, epochs = 5 )

Train on 273 samples, validate on 273 samples
Epoch 1/5
273/273 [==============================] - 21s 77ms/step - loss: 0.6982 - acc: 0.4872 - val_loss: 0.6930 - val_acc: 0.5055
Epoch 2/5
273/273 [==============================] - 19s 70ms/step - loss: 0.6883 - acc: 0.6703 - val_loss: 0.6927 - val_acc: 0.5275
Epoch 3/5
273/273 [==============================] - 19s 69ms/step - loss: 0.6660 - acc: 0.8571 - val_loss: 0.7015 - val_acc: 0.4908
Epoch 4/5
273/273 [==============================] - 19s 68ms/step - loss: 0.5761 - acc: 0.8242 - val_loss: 0.6988 - val_acc: 0.5055
Epoch 5/5
273/273 [==============================] - 19s 71ms/step - loss: 0.5325 - acc: 0.9011 - val_loss: 0.6907 - val_acc: 0.5165
